# Features relations by mobility levels

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import pickle
from matplotlib import dates
import matplotlib.pyplot as plt
import seaborn as sns
import sys; sys.path.append('../')
from src.data_loader.data_loader import *
from src.utils.dates import get_today, lag_date, date2str, str2date, get_format
from src.utils.df_utils import get_date_columns
from src.pandas.align import align_lagged_dates

from scipy.stats import spearmanr

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Time series data
mobility, mobility_date = load_google_mobility()
deaths, deaths_date = load_deaths(join_county_codes=False)
interventions, interventions_date = load_interventions()

# Static data
counties, counties_date = load_counties()

# Processed mobility -> time series
mobility_ts, mobility_ts_date = load_google_mobility_time_series()

m50,_ = load_descartes_m50()

od_mobility = load_od_baseline()

hospitals = load_acute_care(beds=True)

print(f'Mobility Last Updated {mobility_ts_date}')
print(f'Deaths Last Updated {deaths_date}')

# County adjacency edge list
with open('../data/processed/county_adjacency_edge_dict.pkl', 'rb') as f:
    county_adj = pickle.load(f)

Mobility Last Updated 05-08
Deaths Last Updated 05-26


In [3]:
N_DAYS = 9
ONSET_THRESH = 3
GROWTH_THRESH = 0

In [4]:
# Get death dataframe date columns
death_dates = get_date_columns(deaths, return_dtimes=False)

In [5]:
def get_onset_date(row, thresh):
    above = row[row >= thresh]
    if len(above) == 0:
        return np.nan
    else:
        return above.idxmin()

In [8]:
death_data = pd.DataFrame(columns=['FIPS', 'Deaths'])
deaths_df = deaths[['FIPS']+death_dates]

# Get all existing data n_days from the death_thresh death.
for i in range(len(deaths_df)):
    county = deaths_df.iloc[i]

    # Iterate through dates to find first date with deaths >= death thresh
    for j in range(len(death_dates)):
        if county[death_dates[j]] >= ONSET_THRESH:

            # See if this happened n_days or more before current day
            try:
                n_days_later = death_dates[j+N_DAYS]

                # Make sure the data is bug-free
                bad_data=False
                for k in range(n_days):
                    if county[n_days_later] < county[dates[j+k]]:
                        bad_data=True
                if not bad_data:
                    death_data = death_data.append({'FIPS': county['FIPS'], 'Deaths':county[n_days_later]}, ignore_index=True)
                break

            except:
                break